In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
import pandas as pd
import numpy as np


In [2]:
# download these libraries if you don't have them
#%pip install PyPortfolioOpt


In [3]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import cvxpy as cp
from pypfopt import BlackLittermanModel



## Group Assignment
### Team Number: 14
### Team Member Names: Aadya, Furqan, Pranay
### Team Strategy Chosen: Market Beat

### Hello! 
- We hope you’re having a good day so far and that you enjoy running our file! 

#### Note 
 - Our file might encounter a yFinance error that states it is not able to pull a given ticker. In that case, restart the kernel and rerun.  
 
#### Brief overview of our strategy:

After cleaning our csv file for valid tickers, our strategy is to score and then sort our tickers in increasing order based on 4 quantitative metrics - Alpha, Sortino, Momentum, and Beta. This will pick the 10 best stocks (top 10) for our model. We will then use Black-Lattermen Allocation to optimize the weightings of these 10 stocks to beat the benchmark average with long-term range in mind. We will delve more into our reasoning for our choices in the coming markdowns. We are also going to be using 3 year historical data to calculate metrics and optimise.

To get started, input your csv with tickers. We’re sure you know where to input your csv file, but as due diligence, replace _INSERT_CSV_ in the code block below. 

In [4]:

input_tickers = pd.read_csv("Tickers_Example100.csv", names=['Tickers']) #OR any other CSV file with tickers in it
input_tickers

,Tickers
0,Ticker
1,MSFT
2,NVDA
3,GOOGL
4,GOOG
...,...
116,XYZ
117,TICKR
118,NOPE
119,VOID


In [5]:
## PRELIMINARY DATA PROCESSING 
## AND INPUT PROCESSING

## Check if implementation of US and Canadian ticker validation is sufficient/correct
## Should we do it through currency checks, or exchange checks as below?

def validate_ticker(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        if info and 'regularMarketPrice' in info:
            if 'exchange' in info:
                exchange = info['exchange']
                # yfinance uses these exchange codes, so we check against them
                valid_exchanges = ['NMS', 'NGM', 'NYQ','NAD','PCX','BTS','CBQ','ASE', 'TOR', 'TSE', 'Toronto', 'TSXV', 'CNQ', 'CSE','NEO']
                if exchange in valid_exchanges:
                    return True
                else:
                    return False  
            else:
                return False  
        else:
            return False
    except:
        return False

# adding valid tickers to a new list
valid_tickers = []
for ticker in input_tickers['Tickers']:
    if validate_ticker(ticker):
        valid_tickers.append(ticker)
        #print(f"✓ Valid: {ticker}")  
    else:
        print(f"Invalid ticker: {ticker}")

valid_tickers_df = pd.DataFrame(valid_tickers, columns=['Ticker'])
valid_tickers_df


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: TICKER"}}}


Invalid ticker: Ticker
Invalid ticker: SQ


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: CATL.SZ"}}}


Invalid ticker: CATL.SZ


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: RNDM1"}}}


...

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: TICKR"}}}


Invalid ticker: TICKR


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: VOID"}}}


Invalid ticker: VOID
Invalid ticker: nan


,Ticker
0,MSFT
1,NVDA
2,GOOGL
3,GOOG
4,META
...,...
100,META
101,TSLA
102,JPM
103,XYZ


In [6]:
# check if duplicates exist in the csv and remove them
def check_duplicates(df):
    duplicates = df.duplicated()

    for index in duplicates.index:
        if duplicates[index] == True:
            print(f"Removing duplicate: {df.loc[index, 'Ticker']}")
            # remove the row from the dataframe
            df.drop(index, inplace=True)
    return df.reset_index().drop(columns=['index'])

check_duplicates(valid_tickers_df)

Removing duplicate: MSFT
Removing duplicate: NVDA
Removing duplicate: META
Removing duplicate: TSLA
Removing duplicate: JPM


,Ticker
0,MSFT
1,NVDA
2,GOOGL
3,GOOG
4,META
...,...
95,AAL
96,UAL
97,TEST
98,XYZ


In [7]:
## REMOVING STOCKS WITH AVG TRADE VOLUME < 5000 between Oct 1, 2024 and Sep 30, 2025 (drop months with < 18 trading days).
##CHECK to ensure, and filter out stocks that do not meet this criteria

def filter_by_avg_volume(df, start_date="2024-10-01", end_date="2025-09-30", min_avg_volume=5000, min_trading_days=18):
    filtered_tickers = []
    for ticker in df['Ticker']:
        stock = yf.Ticker(ticker)
        
        hist = stock.history(start=start_date, end=end_date)
        hist.index = hist.index.tz_localize(None)

        monthly_groups = hist.groupby(hist.index.to_period("M"))
        valid = True

        # Check each month's average volume and trading days
        for month, group in monthly_groups:
            trading_days = len(group)
            if trading_days >= min_trading_days:
                avg_volume = group['Volume'].mean()
                if avg_volume < min_avg_volume:
                    print(f"Ticker failed volume filter: {ticker} (Avg Volume: {avg_volume:.2f} in {month})")
                    valid = False
                    break
        if valid:
            print(f"Ticker passed volume filter: {ticker}")
            filtered_tickers.append(ticker)
    return pd.DataFrame(filtered_tickers, columns=['Ticker'])

final_tickers_df = filter_by_avg_volume(valid_tickers_df)
final_tickers_df


Ticker passed volume filter: MSFT
Ticker passed volume filter: NVDA
Ticker passed volume filter: GOOGL
Ticker passed volume filter: GOOG
Ticker passed volume filter: META
Ticker passed volume filter: TSLA
Ticker passed volume filter: NFLX
Ticker passed volume filter: INTC
Ticker passed volume filter: ADBE
Ticker passed volume filter: ORCL
Ticker passed volume filter: CRM
Ticker passed volume filter: CSCO
Ticker passed volume filter: IBM
Ticker passed volume filter: AMD
Ticker passed volume filter: MU
Ticker passed volume filter: AMAT
Ticker passed volume filter: ADI
Ticker passed volume filter: INTU
Ticker passed volume filter: NOW
Ticker passed volume filter: SNOW
Ticker passed volume filter: PANW
Ticker passed volume filter: CRWD
Ticker passed volume filter: ZS
Ticker passed volume filter: MDB
Ticker passed volume filter: AFRM
Ticker passed volume filter: UBER
Ticker passed volume filter: LYFT
Ticker passed volume filter: DASH
Ticker passed volume filter: SHOP
Ticker passed volume fi

$TEST: possibly delisted; no price data found  (1d 2024-10-01 -> 2025-09-30) (Yahoo error = "Data doesn't exist for startDate = 1727755200, endDate = 1759204800")


AttributeError: 'Index' object has no attribute 'tz_localize'

In [ ]:
#Downloading history (3y) for valid tickers and market

tickers_list =  valid_tickers_df['Ticker'].tolist()

stock_data= yf.download(tickers_list, period="3y", interval="1d", group_by='ticker', auto_adjust=True, threads=True)

# Download benchmark data (S&P 500 + TSX average)
sp500_data = yf.download('^GSPC', period='3y', auto_adjust=True)
tsx_data = yf.download('^GSPTSE', period='3y', auto_adjust=True)

# checking the 'Close' prices to ensure they are Series
if isinstance(sp500_data, pd.DataFrame):
    sp500 = sp500_data['Close'].squeeze() 
else:
    sp500 = sp500_data

if isinstance(tsx_data, pd.DataFrame):
    tsx = tsx_data['Close'].squeeze()  
else:
    tsx = tsx_data

# Remove timezones
sp500.index = sp500.index.tz_localize(None)
tsx.index = tsx.index.tz_localize(None)


[*********************100%***********************]  32 of 32 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### Stock Selection (following chunks of code before portfolio validation)

#### Overview 
Our first main strategy is to score the valid tickers and rank them based on their scores. The scores are calculated by 4 quantitative metrics - Alpha, Sortino, Momentum, and Beta. From the result of scoring, the top 10 stocks are picked which are our choices for our portfolio (if they don’t conflict with portfolio validation). 

#### Metrics 

From our lessons, we know beta is the measure of risk relative to the market and sharpe ratio is the measure of risk adjusted returns for the total volatility of an asset. Both beta and sharpe can be used to calculate two more quantitative metrics - Alpha and Sortino. 

##### 1) Beta 

Measure of returns compared to the market, which is what we are trying to beat.  

 $$ \beta = \frac{\text{Cov}(R_i, R_m)}{\text{Var}(R_m)} $$

Where:  

- $R_i$ = returns of the individual asset  
- $R_m$ = returns of the overall market (benchmark)  
- $\mathrm{Cov}(R_i, R_m)$ = covariance between the asset’s returns and the market’s returns  
- $\mathrm{Var}(R_m)$ = variance of the market’s returns  


 ##### 2) Jensen's Alpha

Measures the excess returns over the market, taking beta into consideration in the calculation.

$$
\alpha = R_i - \left( R_f + \beta \left( R_m - R_f \right) \right)
$$

Where:

- $R_i$ — the actual return of the stock or portfolio  
- $R_f$ — the risk-free rate (e.g., treasury yield)  
  - We use an annual risk-free rate of **0.04 (4%)**, which all alpha values are based on.  
- $R_m$ — the return of the overall market or chosen benchmark  
- $\beta_i$ — the stock’s sensitivity to market movements (systematic risk) 

Intuitevely:

- $\alpha$ > 0: Means positive returns relative to the market 
- $\alpha$ = 0: The investment's return was in line with the benchmark index
- $\alpha$ < 0: The investment's performance was worse than its benchmark index

##### 3) Sortino Ratio

measures excess return relative to harmful (downside) volatility.
It is a variation of the Sharpe ratio, but it only penalizes negative returns.

$$
\text{Sortino} = \frac{R_p - R_f}{\sigma_d}
$$

Where:

- $R_p$ — the actual return of the portfolio or stock  

- $R_f$ — the risk-free rate (minimum acceptable return)  
  - We use an annual risk-free rate of **0.04**, which we divide by **252 trading days** to obtain the daily $R_f$ used in Sortino.  

- $\sigma_d$ — the downside deviation, which measures only negative volatility  
  (returns that fall below $R_f$)

Intuitevely:

- Sortino > 0: good risk-adjusted performance with limited downside risk  
- Sortino = 0: no excess return relative to downside risk  
- Sortino < 0: poor risk-adjusted performance with significant downside risk  

Note:
- We chose sortino over sharpe as sortino only penalises negative volatility while sharpe penalises both negative and positive volatility, which allows us to pick stocks that show high returns with minimal drawdowns. In essence, having a high alpha and a high sortino in a long-term period should beat the market as it maximises returns and minimises risk, naturally prioritising stocks that are stable long-term outperformers. 


##### 4) Momentum

Our final metric is momentum, which we chose to give us insight into short term movement of the stocks. This is helpful as it highlights the stocks that are doing well right now, allowing us to take that into account to better pick stocks for the scope of this project. 


$$
\text{Momentum} = \left( \frac{R_t}{R_{t-n}} \right) - 1
$$

Where:

- $R_t$ — the price or return of the stock at time $t$
- $R_{t-n}$ — the price or return $n$ periods ago
- $n$ — the lookback period




#### Potential Scenarios

Scenario 1: High Alpha, Low Beta (< 1.0)

 - Stock outperforms with less volatility than market

 - Best case/ideal scenario 

 - Means it is a Defensive stock that somehow beats the market, low risk high return

Scenario 2: High Alpha, High Beta (> 1.5)

 - Stock outperforms but is very volatile

 - Risky but rewarding

 - Also high reward, but higher risk as well. Still viable option to help choose stocks that will beat market

 - Example: Tech stock in a bull market

Scenario 3: Low/Negative Alpha, High Beta

 - Stock underperforms and is volatile

 - Low return relative to market/less than market, AND high volatility so potential further loss on an already underperforming stock.  

 - Worst case 

 - Want to avoid these types of stocks 


WE WANT TO INVEST IN:

Stocks which fall within the first two scenarios, as they are the most ideal to maximize returns, scenario 1 being the most effective. Stocks which fall in scenario 3, will be avoided. 

In [ ]:


#assuming we have a list of valid tickers
#we need a function that simply returns the alpha of a ticker
#then run a for loop that calls that function for every ticker
#while ranking it from best to worst in a list or dictionary maybe
#going to also assume we have the portfolio returns, risk free rate, expected market return, and the portfolio beta


# calculates beta of a stock 
# beta is cov (Rp, Rm) / var (Rm)
def calc_beta(ticker_returns, market_returns):
    covariance = ticker_returns.cov(market_returns)
    variance = market_returns.var()
    beta = covariance / variance
    return beta

#function to calculate alpha of a ticker given the following parameters
#returns the ticker and its alpha in a list
#Good alpha > 0; Means positive returns relative to the market
#Zero alpha (\(\alpha =0\)): The investment's return was in line with the benchmark index, meaning it did not outperform or underperform after adjusting for risk.
#Negative alpha (\(<0\)): The investment's performance was worse than its benchmark index, considering the risk taken. 
def calc_alpha(portfolio_return, risk_free_rate, expected_market_return, portfolio_beta): 
    tick_alpha_calculation = portfolio_return - (risk_free_rate + (portfolio_beta * (expected_market_return-risk_free_rate)))

    return tick_alpha_calculation

# calculating sharpe ratio 
# sharpe ratio of a stock is = (Rp - Rf) / σp
# Good sharpe ratio > 1: indicates good risk-adjusted performance
# Average sharpe ratio ~ 0.5 - 1: indicates moderate risk-adjusted performance
# Poor sharpe ratio < 0.5: indicates poor risk-adjusted performance
# if ticker_returns are daily, then risk free has to be daily as well
def calc_sharpe(ticker_returns, risk_free_rate):
    std = ticker_returns.std()
    mean_return = ticker_returns.mean()
    sharpe_ratio = (mean_return - risk_free_rate) / std
    return sharpe_ratio

# calculating sortino ratio 
# sortino ratio = (Rp - Rf) / σd when σd is the standard deviation of negative asset returns
# sortino ratio focuses only on downside volatility which helps eliminate 
# the risk of downside movements in asset prices
# Intuitively good return per unit of bad volatility
# Good sortino ratio > 0: good risk-adjusted performance with low downside 
# Zero sortino ratio = 0: no excess return relative to downside risk
# Low sortino ratio < 0: poor risk-adjusted performance with high downside risk

def calc_sortino(ticker_returns, risk_free_rate):
    downside_dev = ticker_returns[ticker_returns < 0].std()
    mean_return = ticker_returns.mean()
    sortino_ratio = (mean_return - risk_free_rate) / downside_dev
    return sortino_ratio


##Calculating momentum, shows how much a stock has returned relative to a chosen date/period a while back
#Determining the change in return a given stock has had over a given period of time can aid in picking
#stocks that are trending upwards


def calc_momentum(ticker_returns, period):
    momentum = (ticker_returns.iloc[-1]/ticker_returns.iloc[-period]) - 1
    return momentum





In [ ]:
sp500_aligned, tsx_aligned = sp500.align(tsx, join='inner')

# HERE IS OUR BENCHMARK: Simple arthmetic average of S&P 500 and TSX
#Utilized for our comparitive metrics like Alpha
benchmark_prices = (sp500_aligned + tsx_aligned) / 2
benchmark_returns = benchmark_prices.pct_change().dropna()

risk_free_rate_sortino = 0.04 / 252  # Daily risk-free rate assuming 4% annual
risk_free_rate_alpha = 0.04  # Annual risk-free rate for alpha calculation

years = 3 
market_total_return = (benchmark_prices.iloc[-1] / benchmark_prices.iloc[0]) - 1
market_actual_return = (1 + market_total_return) ** (1/years) - 1  # ANNUALIZED RETURN

## Will build loop though stock_data[] and as it goes through each ticker we can calculate the relevant metrics for those tickers
##Apply created functions for metrics to the tickers while we iterate through the loop

stock_results = []

for ticker in tickers_list:
    stock_close = stock_data[ticker]['Close'].dropna()
    stock_close.index = stock_close.index.tz_localize(None)
    stock_returns = stock_close.pct_change().dropna()

    beta = calc_beta(stock_returns, benchmark_returns)

    
    stock_total_return = (stock_close.iloc[-1] / stock_close.iloc[0]) - 1
    stock_actual_return = (1 + stock_total_return) ** (1/years) - 1


    alpha = calc_alpha(stock_actual_return, risk_free_rate_alpha, market_actual_return, beta)

    sortino = calc_sortino(stock_returns, risk_free_rate_sortino)
    momentum = calc_momentum(stock_close, period=63) #3 months
    
    stock_results.append({
        'Ticker': ticker,
        'Alpha': alpha,
        'Beta': beta,
        'Sortino': sortino,
        'Return': stock_actual_return,
        'Momentum': momentum
    })

    print(f"✓ {ticker}: Alpha={alpha:.4f}, Beta={beta:.2f}, Sortino={sortino:.2f}, Momentum={momentum:.2f}")

results_df = pd.DataFrame(stock_results)

#Sorting results by alpha value, highest to lowest, so we can see ideal stocks to invest in
results_df = results_df.sort_values('Alpha', ascending=False)

print("\n=== Top Stocks by Alpha ===")
results_df.reset_index(drop=True, inplace=True)
results_df
    


    

✓ BCE.TO: Alpha=-0.1857, Beta=0.10, Sortino=-0.07, Momentum=-0.06
✓ CAVA: Alpha=-0.2165, Beta=1.96, Sortino=0.03, Momentum=-0.27
✓ MPW: Alpha=-0.3750, Beta=1.43, Sortino=-0.01, Momentum=0.25
✓ LLY: Alpha=0.3277, Beta=0.69, Sortino=0.10, Momentum=0.44
✓ ADPT: Alpha=0.0149, Beta=2.39, Sortino=0.08, Momentum=0.45
✓ KYIV: Alpha=-0.0333, Beta=0.77, Sortino=0.06, Momentum=0.09
✓ HTFL: Alpha=-0.2001, Beta=1.55, Sortino=0.05, Momentum=-0.02
✓ TCOM: Alpha=0.2316, Beta=0.98, Sortino=0.09, Momentum=0.08
✓ ADBE: Alpha=-0.1674, Beta=1.10, Sortino=0.00, Momentum=-0.09
✓ BDX: Alpha=-0.1504, Beta=0.57, Sortino=-0.02, Momentum=-0.00
✓ NVDA: Alpha=0.9875, Beta=1.90, Sortino=0.18, Momentum=-0.02
✓ ENVX: Alpha=-0.5147, Beta=3.03, Sortino=0.03, Momentum=-0.29
✓ SPY: Alpha=0.0398, Beta=1.08, Sortino=0.08, Momentum=0.02
✓ GOOG: Alpha=0.3014, Beta=1.07, Sortino=0.12, Momentum=0.44
✓ MSFT: Alpha=0.1126, Beta=0.93, Sortino=0.09, Momentum=-0.06
✓ GOOGL: Alpha=0.3035, Beta=1.06, Sortino=0.12, Momentum=0.45
✓ AVGO

,Ticker,Alpha,Beta,Sortino,Return,Momentum
0,PLTR,1.474594,2.415738,0.198009,1.778378,-0.037421
1,NVDA,0.987491,1.901947,0.178275,1.235172,-0.015844
2,AVGO,0.657379,1.768027,0.151730,0.890437,0.143528
3,META,0.565283,1.331265,0.162173,0.750649,-0.211443
4,NFLX,0.383152,1.052260,0.126247,0.538052,-0.149247
5,LLY,0.327689,0.693114,0.097109,0.443373,0.441865
6,TSM,0.310613,1.538362,0.119380,0.518594,0.155864
7,GOOGL,0.303516,1.062246,0.118096,0.459507,0.447948
8,GOOG,0.301375,1.068638,0.117401,0.458064,0.442259
9,TCOM,0.231622,0.982696,0.088129,0.378927,0.075774


#### Scoring 

Since our portfolio’s aim is to beat the market in long-term time span, our sorting strategy scores stocks based on the following weights: 

$$
\text{Score} = 0.30 \alpha + 0.35\,\text{Sortino} + 0.30\,\text{Momentum} + 0.05 \beta
$$


These weightings are optimal for our strategy as alpha and momentum prioritise the growth and the returns of the stock while sortino and beta are both risk adjusted metrics. These weightings are optimal because they allow our model to maximize return generating factors while still maintaining control over downside risk, creating a portfolio that theoretically compounds more than the market in the long run. 

The scoring function generates a dataframe with all tickers sorted from best to last, so the ticker with the lowest score (highest in the sorted list) is the best ticker. Then the top 10 stocks from this dataframe are chosen to be in our initial portfolio. 

In [ ]:
##USING METRICS TO PICK TOP x STOCKS TO INVEST IN

results_df['Alpha_rank'] = results_df['Alpha'].rank(ascending=False)
results_df['Sortino_rank'] = results_df['Sortino'].rank(ascending=False)
results_df['Momentum_rank'] = results_df['Momentum'].rank(ascending=False)

##Beta close to 1.15 is ideal, so we rank based on distance from 1.15
##ascending True means closest to 1.15 gets rank 1, lowest distance to greatest distance
results_df['Beta_rank'] = (results_df['Beta'] - 1.15).abs().rank(ascending=True)

# Score (simple sum - equal weight)
# Different metrics are weighted differently
# Alpha: 30%
# Sortino: 35%
# Momentum: 30%
# Beta: 5%
# This is because sortino is considered more important for risk-adjusted returns
# momentum and alpha are growth indicators 
results_df['Composite_Score'] = (
    (0.30 * results_df['Alpha_rank']) + 
    (0.35 * results_df['Sortino_rank']) + 
    (0.30 * results_df['Momentum_rank']) + 
    (0.05 * results_df['Beta_rank'])
)

# Here is a dataframe with just ranks and composite score
scored_df = results_df[['Ticker', 'Alpha_rank', 'Sortino_rank', 'Momentum_rank', 'Beta_rank', 'Composite_Score']].copy()

# Sorted by composite score
scored_df = scored_df.sort_values('Composite_Score')

scored_df.reset_index(drop=True, inplace=True)

scored_df


,Ticker,Alpha_rank,Sortino_rank,Momentum_rank,Beta_rank,Composite_Score
0,GOOGL,8.0,7.0,2.0,5.0,5.70
1,AVGO,3.0,4.0,8.0,24.0,5.90
2,GOOG,9.0,8.0,3.0,4.0,6.60
3,TSM,7.0,6.0,6.0,16.0,6.80
4,LLY,6.0,10.0,4.0,19.0,7.45
5,NVDA,2.0,2.0,20.0,25.0,8.55
6,PLTR,1.0,1.0,22.0,31.0,8.80
7,TCOM,10.0,12.0,12.0,9.0,11.25
8,META,4.0,3.0,29.0,10.0,11.45
9,JPM,11.0,9.0,18.0,2.0,11.95


In [ ]:
## TOP X STOCKS TO INVEST IN BASED ON METRICS ##
num_stocks_to_invest = 10

top_x_stocks_beforeCHECK = scored_df.head(num_stocks_to_invest)
top_x_stocks_beforeCHECK


,Ticker,Alpha_rank,Sortino_rank,Momentum_rank,Beta_rank,Composite_Score
0,GOOGL,8.0,7.0,2.0,5.0,5.70
1,AVGO,3.0,4.0,8.0,24.0,5.90
2,GOOG,9.0,8.0,3.0,4.0,6.60
3,TSM,7.0,6.0,6.0,16.0,6.80
4,LLY,6.0,10.0,4.0,19.0,7.45
5,NVDA,2.0,2.0,20.0,25.0,8.55
6,PLTR,1.0,1.0,22.0,31.0,8.80
7,TCOM,10.0,12.0,12.0,9.0,11.25
8,META,4.0,3.0,29.0,10.0,11.45
9,JPM,11.0,9.0,18.0,2.0,11.95


### Portfolio Validation Check (Small cap, large cap, industry count) 

The portfolio made from the top 10 stocks is then sent to this validation function that runs the picked tickers against the requirements for small cap, large cap and industry count. If a requirement is not satisfied, the 10th stock is removed and the best one for the certain requirement is added by moving down the list of scored and sorted tickers.  

Note
 - If the given csv does not contain a valid small/large cap. ticker, then the program outputs so and creates a portfolio without that requirement met. 


In [ ]:
### OVERALL PORTFOLIO CHECK. MAKING SURE PORTFOLIO MEETS 

### BASIC REQUIREMENTS:
### 1. NO MORE THEN 40% IN A SINGLE SECTOR
### 2. ONE SMALL CAP
### 3. ONE LARGE CAP


## first going to sort the valid tickers into new lists of largecap, small cap, and industry
## which i can then use to check if my portfolio works

#argument should be the list of all valid tickers
def sort_smallcap(tickers):

    smallcap_tickers = []

    for ticker in tickers:

        tick_info = yf.Ticker(ticker).fast_info
        market_cap = tick_info.get("marketCap")

        if market_cap is not None and market_cap < 2000000000:
            smallcap_tickers.append(ticker)

    return smallcap_tickers
     


def sort_largecap(tickers):

    largecap_tickers = []

    for ticker in tickers:

        tick_info = yf.Ticker(ticker).fast_info
        market_cap = tick_info.get("marketCap")
        
        if market_cap is not None and market_cap > 10000000000:
            largecap_tickers.append(ticker)

    return largecap_tickers



#portfolio argument should be a list of just the X tickers in the portfolio
################################################# IF WE CHANGE NUMBER OF TICKERS WELL HAVE TO REVISE THIS FUNCTION
#final_tickers argument should be a list of all the valid tickers RANKED from best to worst through all of the analysis above
def check_portfolio(portfolio, final_tickers):
    
    print("\n" + "="*60)
    print("PORTFOLIO VALIDATION CHECK")
    print("Check for Small Cap, Large Cap, and Sector Distribution Requirements")
    print("="*60)
    print(f"Initial Portfolio: {portfolio}")
    
    smallcap_tickers = sort_smallcap(final_tickers)
    largecap_tickers = sort_largecap(final_tickers)
    
    print(f"\nAvailable Small Cap Stocks (<$2B): {smallcap_tickers}")
    print(f"Available Large Cap Stocks (>$10B): {largecap_tickers}")

    iteration = 0

    small_cap_exists = True
    large_cap_exists = True

    if len(smallcap_tickers) == 0:
        small_cap_exists = False
    
    if len(largecap_tickers) == 0:
        large_cap_exists = False

    swaps = []


    while True:

        if(not small_cap_exists):
            print("\n NO SMALL CAP stocks available in the valid tickers list. \n Final portfolio WILL NOT have a Small Cap stock.")
            
        if(not large_cap_exists):
            print("\n NO LARGE CAP stocks available in the valid tickers list. \n Final portfolio WILL NOT have a Large Cap stock.")

        iteration += 1
        print(f"\nCheck {iteration}:")
        
        is_smallcap = False
        is_largecap = False
        notfortypercent = True 
        seen_industries = {}
        industry_overforty = None

        for ticker in portfolio:
            if ticker in smallcap_tickers:
                is_smallcap = True
        
            if ticker in largecap_tickers:
                is_largecap = True 

            
            industry = yf.Ticker(ticker).info.get("industry")
            seen_industries[industry] = seen_industries.get(industry, 0) + 1

        # Printing industry breakdown
        print("\nCurrent Industry Breakdown:")
        for industry, amount in seen_industries.items():
            percentage = (amount / num_stocks_to_invest) * 100
            print(f"  {industry}: {amount} stocks ({percentage:.0f}%)")
            if amount / num_stocks_to_invest > 0.4:
                notfortypercent = False
                industry_overforty = industry

        print(f"\n Has Small Cap (<$2B): {'YES' if is_smallcap else 'NO '}")
        print(f" Has Large Cap (>$10B): {'YES' if is_largecap else 'NO '}")
        print(f" No Sector >40%: {'YES' if notfortypercent else f'NO  ({industry_overforty} is over 40%)'}")

        valid_smallcap = (is_smallcap or not small_cap_exists)
        valid_largecap = (is_largecap or not large_cap_exists)

        if valid_smallcap and valid_largecap and notfortypercent:
            print("\n" + "="*60)
            print("PORTFOLIO IS VALID")
            print("="*60)
            print(f"Final Portfolio: {portfolio}")
            return portfolio
            
        if (small_cap_exists and not is_smallcap):
            print("\n Missing Small Cap stock. Will now find replacement")
            for tick in smallcap_tickers:
                if tick not in portfolio:
                    removed = portfolio.pop(-1)
                    portfolio.append(tick)
                    print(f"    Removed: {removed}")
                    print(f"    Added: {tick} (Small Cap)")
                    break
            continue
        elif (not small_cap_exists):
            print("NO SMALL CAP AVAILABLE. CONTINUING with large cap and industry check.")
            

        if (large_cap_exists and not is_largecap):
            print("\n Missing Large Cap stock. Will now find replacement")
            for tick in largecap_tickers:
                if tick not in portfolio:
                    removed = portfolio.pop(-1)
                    portfolio.append(tick)
                    print(f"    Removed: {removed}")
                    print(f"    Added: {tick} (Large Cap)")
                    break

            continue
        elif (not large_cap_exists):
            print("NO LARGE CAP AVAILABLE. CONTINUING with industry check.")


        #logic for this is to check if there is an industry over 40, and if there is we know we have to repalce some tickers
        #so we take the last occurence of that ticker in the portfolio, since thats the worst one, and take it out and append the first
        #ticker in the rest of our list, assuming it was already not in our ticker(to stop infintly swapping the same 2 or 3 tickers) 
        if not notfortypercent and industry_overforty is not None:
            print(f"\n Sector '{industry_overforty}' is over 40%. Will find a replacement to meet requirment")
            
            worstposition = -1
            removeworst = None
            swapwith = None

            for tick in portfolio:
                tick_industry = yf.Ticker(tick).info.get("industry")
                if tick_industry == industry_overforty:
                    position = final_tickers.index(tick)
                    if position > worstposition:
                        worstposition = position
                        removeworst = tick

            for tick in final_tickers:
                if tick not in portfolio and tick != removeworst and tick not in swaps:
                    tick_industry = yf.Ticker(tick).info.get("industry")
                    if tick_industry != industry_overforty:
                        swapwith = tick
                        break

            if removeworst is not None and swapwith is not None:
                swap_industry = yf.Ticker(swapwith).info.get("industry")
                portfolio.remove(removeworst)
                portfolio.append(swapwith)
                print(f"    Removed: {removeworst} (Sector: {industry_overforty})")
                print(f"    Added: {swapwith} (Sector: {swap_industry})")

                swaps.append(removeworst)
                swaps.append(swapwith)

            continue

        else:
            return portfolio
        



In [ ]:
full_1st = scored_df['Ticker'].tolist()
portfolio_lst = top_x_stocks_beforeCHECK['Ticker'].tolist()

top_x_stocks = check_portfolio(portfolio_lst, full_1st)

##THE LIST HERE should go to the variable top_x_tickers below
#Whatever list is outputted after the check for volume and industry, should move on to the optimization code
#as a list and set equal to top_x_tickers



PORTFOLIO VALIDATION CHECK
Check for Small Cap, Large Cap, and Sector Distribution Requirements
Initial Portfolio: ['GOOGL', 'AVGO', 'GOOG', 'TSM', 'LLY', 'NVDA', 'PLTR', 'TCOM', 'META', 'JPM']

Available Small Cap Stocks (<$2B): ['QH', 'ENVX']
Available Large Cap Stocks (>$10B): ['GOOGL', 'AVGO', 'GOOG', 'TSM', 'LLY', 'NVDA', 'PLTR', 'TCOM', 'META', 'JPM', 'NFLX', 'RY', 'MSFT', 'TSLA', 'BRK-B', 'BRK-A', 'COST', 'JNJ', 'BAC', 'MA', 'XOM', 'BDX', 'ADBE', 'BCE.TO']

Check 1:

Current Industry Breakdown:
  Internet Content & Information: 3 stocks (30%)
  Semiconductors: 3 stocks (30%)
  Drug Manufacturers - General: 1 stocks (10%)
  Software - Infrastructure: 1 stocks (10%)
  Travel Services: 1 stocks (10%)
  Banks - Diversified: 1 stocks (10%)

 Has Small Cap (<$2B): NO 
 Has Large Cap (>$10B): YES
 No Sector >40%: YES

 Missing Small Cap stock. Will now find replacement
    Removed: JPM
    Added: QH (Small Cap)

Check 2:

Current Industry Breakdown:
  Internet Content & Information: 3

In [ ]:

#We sorted and picked the top 10 tickers for our portfolio based on our composite score above
# So Here we get actual metric values for top x stocks from results_df
top_x_metrics = results_df[results_df['Ticker'].isin(top_x_stocks)].copy()

prices_df = pd.DataFrame()
for ticker in top_x_stocks:
    stock_close = stock_data[ticker]['Close'].dropna()
    stock_close.index = stock_close.index.tz_localize(None)
    prices_df[ticker] = stock_close

prices_df = prices_df.dropna()
prices_df


,GOOGL,AVGO,GOOG,TSM,LLY,NVDA,PLTR,TCOM,META,QH
Date,,,,,,,,,,
2022-11-22,96.385223,50.355572,96.668480,78.542152,352.409424,16.018753,7.220000,26.640530,110.757217,139.500000
2022-11-23,97.785568,50.696621,98.148338,77.999763,353.229736,16.499178,7.370000,26.978886,111.552315,130.500000
2022-11-25,96.792412,50.342262,96.936638,77.457375,356.676880,16.250475,7.280000,25.824497,110.727402,127.349998
2022-11-28,95.392090,49.610752,95.595810,75.382957,357.184662,15.808007,7.160000,27.974051,108.113510,115.199997
2022-11-29,94.537956,49.532852,94.791313,75.487633,355.407410,15.620234,7.080000,29.058779,108.789337,123.300003
...,...,...,...,...,...,...,...,...,...,...
2025-11-17,285.019989,342.649994,285.600006,282.010010,1021.700012,186.600006,171.250000,70.889999,602.010010,0.975000
2025-11-18,284.279999,340.500000,284.959991,277.910004,1030.050049,181.360001,167.330002,72.440002,597.690002,1.040000
2025-11-19,292.809998,354.420013,292.989990,282.369995,1049.599976,186.520004,165.419998,72.449997,590.320007,0.971000


In [ ]:
print(f"\n{'='*60}")
print("BLACK-LITTERMAN PORTFOLIO OPTIMIZATION")
print(f"{'='*60}")

# For the Black-Litterman Model, we require a market baseline return
#Our basis for the market baseline will be the average return of the S&P 500 and TSX over the past 5 years
#This will give us a reasonable expectation of market returns to build our views upon
#It shows the avergae growth and trends of the benchmark average/markets over the period we are interested in
#Giving us an relatively accurate average to base our views on
years = 3
market_total_return = (benchmark_prices.iloc[-1] / benchmark_prices.iloc[0]) - 1
market_baseline = (1 + market_total_return) ** (1/years) - 1

print(f"Market Baseline Annual Return: {market_baseline:.2%}") 

viewdict = {}
confidences = []


#The black litterman model requires us to set views and confidences for each stock
#We will use alpha and momentum to set our views on expected returns for each stock
#We will use alpha and sortino to set our confidence in those views

#The optimization technqiue/model is meant to be built around our outlook for the market and the stocks
#To effectively decide the most optimal weights for each stock in the portfolio
#To ensure accurate views and confidences provided from our end, we are using the quantivative metrics we calculated earlier on each stock
#By using hard data, from metrics like Alpha, Sortino and Momentum, we are able to set, a rough reasonable calculation on how the stock will perform for the optimization
# These calculations can ensure a more accurate and effective optimization process

for _, row in top_x_metrics.iterrows():
    ticker = row['Ticker']
    alpha_annual = row['Alpha']
    momentum_3mo = row['Momentum']
    momentum_annual = (1 + momentum_3mo) ** 4 - 1
    
    expected_return = market_baseline + alpha_annual + (0.5 * momentum_annual)
    expected_return = np.clip(expected_return, 0.05, 0.30)
    viewdict[ticker] = expected_return
    
    alpha_norm = row['Alpha'] / top_x_metrics['Alpha'].max()
    sortino_norm = row['Sortino'] / top_x_metrics['Sortino'].max()
    combined = (alpha_norm + sortino_norm) / 2
    confidence = 0.5 + combined * 0.5
    confidences.append(confidence)

S = risk_models.sample_cov(prices_df)

bl = BlackLittermanModel(
    S, 
    absolute_views=viewdict, 
    view_confidences=confidences,
    pi='equal'  
)

ret_bl = bl.bl_returns()
S_bl = bl.bl_cov()

# Optimize
risk_free = 0.04
lower_bound_weight = (100 / (2 * num_stocks_to_invest)) / 100 # 5% for 10 stocks

ef = EfficientFrontier(ret_bl, S_bl, weight_bounds=(lower_bound_weight, 0.15)) #HERE WE SET BOUNDS FOR THE WEIGHT REQURMENTS AS PER THE ASSIGNMENT: 5-15% per stock
weights = ef.max_sharpe(risk_free_rate=risk_free)
cleaned_weights = ef.clean_weights()

performance = ef.portfolio_performance(
    verbose=False, 
    risk_free_rate=risk_free 
)

expected_return, expected_volatility, sharpe_ratio = performance

weights_df = pd.DataFrame({
    'Ticker': list(cleaned_weights.keys()),
    'Weight': list(cleaned_weights.values())
}).sort_values('Weight', ascending=False)

weights_df['Weight'] = weights_df['Weight'] / weights_df['Weight'].sum()

weights_df['Weight'] = weights_df['Weight'].round(4)

print(f"\n{'='*60}")
print("OPTIMIZED PORTFOLIO WEIGHTS")
print(f"{'='*60}")
print(weights_df.to_string(index=False))

weights_sum = weights_df['Weight'].sum()

print(f"Sum: {weights_sum}")

print(f"\nExpected Annual Return:    {expected_return:.2%}")
print(f"Expected Annual Volatility: {expected_volatility:.2%}")
print(f"Expected Sharpe Ratio:      {sharpe_ratio:.2f}")



BLACK-LITTERMAN PORTFOLIO OPTIMIZATION
Market Baseline Annual Return: 14.92%

OPTIMIZED PORTFOLIO WEIGHTS
Ticker  Weight
 GOOGL  0.1500
  GOOG  0.1500
   LLY  0.1500
  TCOM  0.1500
  META  0.1123
   TSM  0.0877
  AVGO  0.0500
  NVDA  0.0500
  PLTR  0.0500
    QH  0.0500
Sum: 1.0

Expected Annual Return:    24.21%
Expected Annual Volatility: 25.75%
Expected Sharpe Ratio:      0.78


In [ ]:

### COMMENT OUT THIS CODE BLOCK OR THE ONE BEFORE FOR WHICHEVER IS APPROPRIATE
### FINAL CODE PORTFOLIO GENERATION
### ASSUMING MAX FEE FOR THE ENTIRE PORTFOLIO IS 2.15 * NUM STOCKS BOUGHT

### AFTER MUCH CONSIDERATION. THIS IS THE CORRECT APPROACH...

money_cad = 1000000
maxfee_usd = 2.15 
numstocks = num_stocks_to_invest

#calculating fees for a stock

def calc_fee(shares):
    fee_usd = min(2.15, shares * 0.001)
    usd_to_cad = yf.Ticker("CAD=X").fast_info["lastPrice"]
    return fee_usd * usd_to_cad

##takes dataframe with columns "Ticker" and "Weight" as a DECIMAL
def produce_finaldf(df_tickers_and_weight):


    #get exchange rates for both directions
    usd_to_cad = yf.Ticker("CAD=X").fast_info["lastPrice"]
    cad_to_usd = 1 / usd_to_cad



    ### we cant know the real fee until we know the real number of shares
    ### and we cant know the real number of shares until we know how much money we are actually investing
    ### so there is a loop here where shares depend on fees and fees depend on shares

    ### so well do it twice
    ### first we pretend we invest the full 1 million before fees so we can calculate fake shares for each stock and fees for each stock and we get fees for the entire portfolio
    ### now that we  know the real total fee we subtract it from the 1 million to get the real investable amount
    ### then we calculate the real shares using that corrected amount
    ### the fees are from the first run still but because our max fees are 21.5 its safe to use the fees from the first one because there shares and fees wont be too different



    tickers = df_tickers_and_weight["Ticker"].tolist()
    weights = df_tickers_and_weight["Weight"].tolist()

    fees_listed = []
    shares_pre_fee = []
    price_listed = []
    currency_listed =[]

    for indx, row in df_tickers_and_weight.iterrows():

        ticker = row["Ticker"]
        weight_decimal = row["Weight"]

        spend_amnt_cad = money_cad * weight_decimal

        info = yf.Ticker(ticker).fast_info
        price = info["lastPrice"]
        currency = info.get("currency", "USD")

        if currency == "USD":
            price_cad = price * usd_to_cad
        else:
            price_cad = price

        shares = spend_amnt_cad / price_cad

        shares_pre_fee.append(shares)
        price_listed.append(price)
        currency_listed.append(currency)

        #fees in cad
        fees_listed.append(calc_fee(shares))

    total_fees_final = sum(fees_listed)
    total_investable_cad = money_cad - total_fees_final

    #final calcualtions for df

    final_tickers = []
    final_price = []
    final_currency = []
    final_shares = []
    final_value_cad = []
    final_weights = []
    
    for i in range(len(tickers)):
        
        ticker = tickers[i]
        weight_decimal = weights[i]
        price = price_listed[i]
        currency = currency_listed[i]

        spend_amnt_cad = total_investable_cad * weight_decimal

        if currency == "USD":
            price_cad = price * usd_to_cad
        else:
            price_cad = price

        shares = spend_amnt_cad / price_cad
        value_cad = shares * price_cad

        final_tickers.append(ticker)
        final_price.append(round(price, 2))
        final_currency.append(currency)
        final_shares.append(shares)  
        final_value_cad.append(round(value_cad,2))
        final_weights.append(round(weight_decimal * 100,2))




    df_final = pd.DataFrame({
        "Ticker": final_tickers,
        "Price": final_price,
        "Currency": final_currency,
        "Shares": final_shares,
        "Value (CAD)": final_value_cad,
        "Weight (%)": final_weights})
    
    portfolio_val = sum(final_value_cad)
    total_spend_withfees = portfolio_val + total_fees_final
    

    
    print("\n\nTotal Portfolio Value (CAD): $", round(portfolio_val,2))
    print("Fee Charged (CAD): $", round(total_fees_final,2))
    print("Total Spent Including Fees (CAD): $", round(total_spend_withfees,2))


    return df_final



#print(weights_df)
Portfolio_Final = produce_finaldf(weights_df)
#Portfolio_Final.index = range(1, len(Portfolio_Final)+1)
df_ticker_shares = Portfolio_Final[["Ticker", "Shares"]].copy()



total_value = Portfolio_Final["Value (CAD)"].sum()
total_weight = Portfolio_Final["Weight (%)"].sum()

#total value and total weight
summary_rows = pd.DataFrame([
    {
        "Ticker": "TOTAL VALUE",
        "Price": "",
        "Currency": "",
        "Shares": "",
        "Value (CAD)": round(total_value, 2),
        "Weight (%)": ""
    },
    {
        "Ticker": "TOTAL WEIGHT",
        "Price": "",
        "Currency": "",
        "Shares": "",
        "Value (CAD)": "",
        "Weight (%)": round(total_weight, 2)
    }])


Portfolio_Final = pd.concat([Portfolio_Final, summary_rows], ignore_index=True)
Portfolio_Final.index = range(1, len(Portfolio_Final) + 1)

display(Portfolio_Final)

df_ticker_shares.to_csv('Stocks_Group_14.csv', index=False)
print("CSV EXPORT COMPLETE")
 




Total Portfolio Value (CAD): $ 999992.41
Fee Charged (CAD): $ 7.58
Total Spent Including Fees (CAD): $ 999999.99


,Ticker,Price,Currency,Shares,Value (CAD),Weight (%)
1,GOOGL,299.66,USD,355.085123,149998.86,15.0
2,GOOG,299.65,USD,355.096984,149998.86,15.0
3,LLY,1059.7,USD,100.410318,149998.86,15.0
4,TCOM,69.85,USD,1523.33302,149998.86,15.0
5,META,594.25,USD,134.054243,112299.15,11.23
6,TSM,275.06,USD,226.173728,87699.34,8.77
7,AVGO,340.2,USD,104.257109,49999.62,5.0
8,NVDA,178.88,USD,198.279678,49999.62,5.0
9,PLTR,154.85,USD,229.049198,49999.62,5.0
10,QH,0.92,USD,38565.041522,49999.62,5.0


CSV EXPORT COMPLETE


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.